In [1]:
import pandas as pd
import pickle
from scipy.stats import poisson

In [2]:
tablas = pickle.load(open('grupos','rb'))
mundiales = pd.read_csv('mundiales_clean.csv')
mundial_qatar = pd.read_csv('mundialQatar_clean.csv')

In [3]:
mundiales

,local,visitante,año,resultado_local,resultado_visitante,Total_Goals
0,France,Mexico,1930,4,1,5
1,Argentina,France,1930,1,0,1
2,Chile,Mexico,1930,3,0,3
3,Chile,France,1930,1,0,1
4,Argentina,Mexico,1930,6,3,9
...,...,...,...,...,...,...
895,Russia,Croatia,2018,2,2,4
896,France,Belgium,2018,1,0,1
897,Croatia,England,2018,2,1,3
898,Belgium,England,2018,2,0,2


In [6]:
poder_local = mundiales[['local','resultado_local','resultado_visitante']]
poder_visitante = mundiales[['visitante','resultado_visitante','resultado_local']]

In [8]:
poder_local.rename(columns={'local':'equipo','resultado_local':'goles_favor','resultado_visitante':'goles_contra'},inplace=True)
poder_visitante.rename(columns={'visitante':'equipo','resultado_visitante':'goles_favor','resultado_local':'goles_contra'},inplace=True)

C:\Users\Andrés Dardex\AppData\Local\Temp\ipykernel_25568\1956214220.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  poder_local.rename(columns={'local':'equipo','resultado_local':'goles_favor','resultado_visitante':'goles_contra'},inplace=True)
C:\Users\Andrés Dardex\AppData\Local\Temp\ipykernel_25568\1956214220.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  poder_visitante.rename(columns={'visitante':'equipo','resultado_visitante':'goles_favor','resultado_local':'goles_contra'},inplace=True)


In [9]:
poder_local

,equipo,goles_favor,goles_contra
0,France,4,1
1,Argentina,1,0
2,Chile,3,0
3,Chile,1,0
4,Argentina,6,3
...,...,...,...
895,Russia,2,2
896,France,1,0
897,Croatia,2,1
898,Belgium,2,0


In [13]:
poder_equipo = pd.concat([poder_local,poder_visitante],ignore_index=True).groupby('equipo').mean()

In [16]:
def predecir_puntos(local, visitante):
    if local in poder_equipo.index and visitante in poder_equipo.index:
        valor_local = poder_equipo.at[local,'goles_favor'] * poder_equipo.at[visitante,'goles_contra']
        valor_visitante = poder_equipo.at[visitante,'goles_favor'] * poder_equipo.at[local,'goles_contra']
        probabilidad_local, probabilidad_visitante, probabilidad_empate = 0,0,0
        for i in range(0,10):
            for j in range(0,10):
                if i > j:
                    probabilidad_local += poisson.pmf(i,valor_local) * poisson.pmf(j,valor_visitante)
                elif i < j:
                    probabilidad_visitante += poisson.pmf(i,valor_local) * poisson.pmf(j,valor_visitante)
                else:
                    probabilidad_empate += poisson.pmf(i,valor_local) * poisson.pmf(j,valor_visitante)
        puntos_local = probabilidad_local * 3 + probabilidad_empate
        puntos_visitante = probabilidad_visitante * 3 + probabilidad_empate
        return (puntos_local,puntos_visitante)
    else:
        return(0,0) #para equipos que participan por primera vez en el mundial

In [23]:
mundial_qatar_grupos = mundial_qatar[:48].copy()
mundial_qatar_octavos = mundial_qatar[48:56].copy()
mundial_qatar_cuartos = mundial_qatar[56:60].copy()
mundial_qatar_semis = mundial_qatar[60:62].copy()
mundial_qatar_final = mundial_qatar[62:].copy()